In [1]:
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
#from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.preprocessing import MinMaxScaler


In [3]:
data = pd.read_csv('AMD.csv')


In [4]:
data

,ticker,date,open,high,low,close
0,AMD,1980-03-17,0.00,3.30,3.12,3.15
1,AMD,1980-03-18,0.00,3.12,2.94,3.03
2,AMD,1980-03-19,0.00,3.08,3.02,3.04
3,AMD,1980-03-20,0.00,3.06,3.01,3.01
4,AMD,1980-03-21,0.00,3.02,2.91,2.92
...,...,...,...,...,...,...
10860,AMD,2023-04-13,92.79,93.16,91.83,92.09
10861,AMD,2023-04-14,91.82,92.97,90.50,91.75
10862,AMD,2023-04-17,90.23,90.69,88.30,89.87
10863,AMD,2023-04-18,91.61,92.16,89.33,89.78


In [5]:
data = data.filter(['close'])


In [6]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,close
9865,26.81
9866,28.29
9867,28.22
9868,27.42
9869,26.66
...,...
10860,92.09
10861,91.75
10862,89.87
10863,89.78


In [7]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data.values.reshape(-1,1))

In [8]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

#dataScale = data
train_percent = 0.8
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [9]:

data = train_data

#dataScale = data
train_percent = 0.8
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]


In [10]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 7
size_predict = 1
stepWindow = 1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [11]:
start_time = time.time()


number_list = [5,6,8,9]

num_hidden_layer = number_list[0] # ví dụ chọn số 11

In [12]:

# Tạo mô hình CNN
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(size_window, 1)))
model.add(Conv1D(filters=32, kernel_size=2, activation='relu'))

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
for i in range(num_hidden_layer): #Thêm num_hidden_layer lớp ẩn với 64 đơn vị mỗi lớp
    model.add(Dense(units=100, activation='relu')) #Tham số chỉ định số lượng tế bào thần kinh trong lớp
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

# Compile mô hình với Adam optimizer và Mean Squared Error loss
model.compile(loss='mse', optimizer='adam')


In [13]:
history = model.fit(X_train, y_train, epochs=8000, batch_size=8)

Epoch 1/8000
79/79 [==============================] - 2s 1ms/step - loss: 0.0537
Epoch 2/8000
79/79 [==============================] - 0s 1ms/step - loss: 0.0135
Epoch 3/8000
79/79 [==============================] - 0s 1ms/step - loss: 0.0063
Epoch 4/8000
79/79 [==============================] - 0s 1ms/step - loss: 0.0053
Epoch 5/8000
79/79 [==============================] - 0s 1ms/step - loss: 0.0048
Epoch 6/8000
79/79 [==============================] - 0s 1ms/step - loss: 0.0040
Epoch 7/8000
79/79 [==============================] - 0s 1ms/step - loss: 0.0033
Epoch 8/8000
79/79 [==============================] - 0s 1ms/step - loss: 0.0032
Epoch 9/8000
79/79 [==============================] - 0s 1ms/step - loss: 0.0033
Epoch 10/8000
79/79 [==============================] - 0s 1ms/step - loss: 0.0036
Epoch 11/8000
79/79 [==============================] - 0s 1ms/step - loss: 0.0027
Epoch 12/8000
79/79 [==============================] - 0s 1ms/step - loss: 0.0026
Epoch 13/8000
79/79 [====

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.reshape(y_pred, (-1, 1))
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)

print('MSE: ', mse)
print('RMSE: ', rmse)
print('MAPE: ', mape)

In [ ]:
len(data)